<a href="https://colab.research.google.com/github/Yuhao-Zhang-SV/spikeinterface-yuhaolib/blob/main/seqConcatenateRHDFiles25GB_variable_bits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Four Cells are Start-up Code

This cell mounts your google drive at the location '/content/drive' on the virtual machine. After executing this, you will be able to see your google drive folder by expanding the file icon in the left panel. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This cell adds the Chamanzar Lab libraries from '/lib' to the system path, so that they can be imported and used by the virtual machine runtime.

In [ ]:
import sys
libdir = "/content/drive/MyDrive/EPhys Recordings/lib"
sys.path.insert(1, libdir)

#Jay Note: not sure how the imports were working before, but I got it working 
# by adding the intan parser to the lib dir.
intandir = "/content/drive/MyDrive/EPhys Recordings/lib/intan"
sys.path.insert(1, intandir)

This cell imports the third-party libraries that are used later on in this file. 

In [ ]:
# Concatenating channels
from intan.load_intan_rhd_format import read_data
import pandas as pd
import numpy as np
import statistics
from scipy.fft import fft
import matplotlib.pylab as plt
import os
from multiprocessing import Pool, get_context, set_start_method
from functools import partial
from IPython.core.debugger import set_trace
import tracemalloc

This cell defines all of the necessary concatenation functions

In [ ]:

# put r in front of string to denote raw string or it is interpreted as unicode
# also equivalent to dirpath.osgetcwd()

# returns list of file names that end with ".rhd" in the current directory
def listFiles(path, recursive = True): #adapted from 112 website
    if (os.path.isdir(path) == False):
        # base case:  not a folder, but a file, so return singleton list with its path
        if (path.endswith('.rhd')):
            return [path]
        else:
            return []
    if recursive == True:
              # recursive case: it's a folder, return list of all paths
        files = [ ]
        for filename in os.listdir(path):
            files += listFiles(path + os.sep + filename)
        
        return files

# returns list of base file names (truncated path) for an inputted list of files with full path names
def fileBaseOnly(fileList):
    result = []
    for fileName in fileList:
        result.append(os.path.basename(fileName))
    return result

# below is general version
def getTime(fileName):
    fileName1 = fileName.split(".")[-2]
    fileInfo = fileName1.split('_')
    print("fileInfo", fileInfo)
    #print(seconds)
    seconds = int((fileInfo[-1])[4:])
    minutes = int((fileInfo[-1])[2:4])
    hours = int((fileInfo[-1])[0:2])
    time = hours*3600 + minutes*60 + seconds
    return time

# Get the name in format: {name}_date_time.rhd
def getName(fileName):
    fileName1 = fileName.split(".")[0]
    # get everything before date,time
    name = '_'.join(fileName1.split('_')[:-2])
    return name

def removePath(fileName):
  fn = fileName.split("/")[-1]
  return fn

# function that checks size of 2D array - used for assert in sortFiles
def sizeOf(list2D):
    result = 0
    for batch in list2D:
        for file in batch:
            result +=1
    return result
        
        
# function that takes a .rhd file list and outputs 2-d array, groups lists of rhd files that belong in same experiment batch
def sortFiles(fileList):
    # this may need to be modified to account for files that are out of order
    print("fileList", fileList)
    assert(len(fileList) >= 1)
    result = []
    batch = []
    batch.append(fileList[0])
    for i in range(len(fileList)-1):
        file1 = fileList[i]
        file1Time = getTime(file1)
        file2 = fileList[i+1]
        file2Time = getTime(file2)
        timeDiff = file2Time - file1Time
        ## Filenames for different recording sessions may also be different
        file1Name = getName(file1)
        file2Name = getName(file2)
        if ((58 <= timeDiff <= 62) or (298 <= timeDiff <= 302))and (file1Name == file2Name): #300 is for 5 min files
            batch.append(file2)
        else:
            result.append(batch)
            batch = []
            batch.append(file2)
    
    result.append(batch)
    # checks to make sure all input .rhd files are grouped into a batch
    assert(sizeOf(result) == len(fileList))
    return result

# Helper function for parallelized processing
def processFile(filename, totalChanNum= 32, includeADC= False, includeDIO= False):
      converted = read_data(filename) # converted should be the numpy array
      currFileArray = converted['amplifier_data']
      channel_info = converted['amplifier_channels']

      impedances = [(c['electrode_impedance_magnitude'],
                    c['electrode_impedance_phase']) for c in channel_info]

      channel_numbers = [c['chip_channel'] for c in channel_info]
      channel_offset = np.array([int(c['port_number']) -2 for c in channel_info]) *32
      channel_numbers = list(np.array(channel_numbers)+ channel_offset)

      if includeADC:
          assert 'board_adc_data' in converted, "includeADC set to True, but file contains no adc data"
          currFileArray =  np.append(currFileArray, converted['board_adc_data'], axis = 0)

      if includeDIO:
          assert 'board_dig_in_data' in converted, "includeDIO set to True, but file contains no dio data"
          digdata = converted['board_dig_in_data']
          currFileArray =  np.append(currFileArray, digdata, axis = 0) 
          
      #print(len(currFileArray))
      #assert(len(currFileArray) == totalChanNum)
      print("currFileArray size", currFileArray.shape)


      return (currFileArray, channel_numbers, impedances)

# function takes in a 2d list of file names, allData[batch][all files in batch]
# where batch represents uninterrupted experiment, and files represents each rhd data file in that experiment run
def sequentialProcessData(allData, fileName, totalChanNum = 32, includeADC= False, includeDIO =False, dtype = np.float64):
    print("allData", allData)
    for batch in allData:
        print(batch[0])
        f = partial(processFile, totalChanNum = totalChanNum, includeADC=includeADC, includeDIO=includeDIO)

        batchResult, channel_numbers, impedances = None,None,None

        batchResult_index = 0
        for element in batch:
          data, channel_numbers,impedances = f(element)

          # do conversion here to reduce footprint: 
          data = data.astype(dtype)

          if batchResult is None:
            # batchResult doesn't exist yet, so this is the first element
            # let's initialize it

            # We can use the number of items in the batch and the size of the
            # individual batch element to allocate the proper array size
            num_elems = len(batch)
            arr_ax1, arr_ax2 = data.shape
            batch_shape = (arr_ax1, arr_ax2*num_elems)
            
            batchResult = np.empty(batch_shape, dtype= dtype)

          # fill batchResult with the current data chunk
          # we know the start and stop indices based on the chunk size
          _, data_len = data.shape
          start_index,stop_index = batchResult_index, batchResult_index + data_len

          # do the assignment
          batchResult[:, start_index:stop_index] = data

          # update the index for the next run
          batchResult_index += data_len
          
        # trim the ndArray to the right size, since the last chunk can be a
        # partial chunk

        ax1_len, ax2_len = batchResult.shape
        trimmed_shape = (ax1_len, batchResult_index)
        batchResult.resize(trimmed_shape)

        # slice out DIO segment
        batchName = removePath(getName(batch[0]))
        if includeDIO:
            print('chop out DIO')
            diodata = batchResult[-1,:]
            plt.plot(diodata)
            diodata.tofile(fileName+batchName+'trigger.RAW')
            batchResult = batchResult[:-1, :]
        assert(len(batchResult) == totalChanNum)
        
        batchResult.tofile(fileName + batchName + dtype.__name__ + ".RAW")
        print(impedances)
        print(channel_numbers)
        np.savetxt(fileName+batchName+'_impedances.csv', impedances, delimiter = ',')
        np.savetxt(fileName+batchName+'_chananels.csv', channel_numbers, delimiter = ',')
        print("CREATED FILE", fileName + batchName +  dtype.__name__+ ".RAW")

def main(base_dir, fileName, totalChanNum, includeADC=False, includeDIO=False, dtype = np.float64):
    # define base_dir as the folder in which the relevant data files are held
    RHDfileListRaw = listFiles(base_dir) #return list of full file path for files that end in ".rhd" in current directory
    sortedRHDfileListRaw = sorted(RHDfileListRaw) #alphabetize .rhd file list
    print("alphabetized RHDfileListRaw", sortedRHDfileListRaw) #sorts list of all RHD files in directory alphabetically
    # note, if you're specifying directory, don't use fileBaseOnly, because you need full path name
    allDataSorted = sortFiles(sortedRHDfileListRaw) #returns 2-D list of lists of RHDfile path names grouped together in experiment batches 
    print("grouped .rhd files", allDataSorted)
    print("num groups", len(allDataSorted))

    # for compressed data, say dtype=np.float32, if this optional parameter is removed, it will default to float64
    # parallelProcessData(allDataSorted, fileName, totalChanNum, includeADC=includeADC, includeDIO=includeDIO, dtype = dtype)
    sequentialProcessData(allDataSorted, fileName, totalChanNum, includeADC=includeADC, includeDIO=includeDIO, dtype = dtype)

The cell below does all of the work of concatenating RHD files. You should set 'base_dir' to the folder that contains all of the files. Then, set totalChanNum to the number of channels in the recording. 

Finally, the fileName extension doesn't matter, as long as it is entered correctly later on in the process. 

In [ ]:
### Edit the variables in this cell
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/20210702/M1_P1_AP-2ML1-5DV2-5-210702-185639'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/20210702/M1_P1_AP-2ML1-5DV2-5-210702-185739(Skip 1st Min)'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Zabir_Rodent_MI_ 07.16.21/Depth 3.25 mm'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Jay Rodent in vivo/07-09-2021'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Zabir_Rodent_MI_07.16.21/Depth 3.45 mm'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Jay Rodent in vivo/08-19-2021/M1 ReaChr/continuous_40ms_410mV_0.25hz'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/NHP_10-05-21/Walter_20211005/Walter_20211005_1335_z4250'
#base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/NHP_10.05.21/Walter_20211005/Walter_20211005_1340_z4000'
#base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/NHP_10-05-21/Walter_20211005/Walter_20211005_1340_z4000'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/NHP_10-05-21/Walter_20211005/Walter_20211005_1325_z4500'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/NHP_10-05-21/Walter_20211005/Walter_20211005_1340_z4000'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Mice-long-steeltrode-12.21.21/3_211221_150605'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Mice-long-steeltrode-12.21.21/2_211221_140524'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Rodent dura 20220418/DV3'
base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500'
# base_dir = '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Jay Rodent in vivo/08-19-2021/M1 ReaChr/continuous_40ms_410mV_0.25hz'


fileName = base_dir + "batch" # change this to the name of the .RAW stitched file you want saved
totalChanNum = 56#49#64 # total number of channels in your recording

# allows you to specify the datatype. If you don't, it defaults to float64
main(base_dir, fileName, totalChanNum, includeADC=False, includeDIO=False, dtype = np.float32)

alphabetized RHDfileListRaw ['/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500/Walter_20220422_1017_zm4500_220422_101619.rhd', '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500/Walter_20220422_1017_zm4500_220422_101719.rhd', '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500/Walter_20220422_1017_zm4500_220422_101819.rhd', '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500/Walter_20220422_1017_zm4500_220422_101919.rhd', '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500/Walter_20220422_1017_zm4500_220422_102019.rhd', '/content/drive/My Drive/EPhys Recordings/Chamanzar Lab Recording/Zabir/Walter 20220422/Walter_20220422_1017_zm4500/Walter_20220422_1017_zm4500_220422_